In [4]:
#Step 1. 필요한 모듈과 라이브러리를 import 합니다

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys

import math
import numpy  
import pandas as pd  
import xlwt

#Step 2. 필요한 정보를 입력 받습니다
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))

page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

f_name = input('3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fc_name = input('4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fx_name = input('5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/main/main.html')
time.sleep(2)

#Step 4. 검색창의 이름을 찾아서 검색어를 입력 후 검색을 진행합니다
driver.find_element_by_id("btnSearch").click()
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
driver.find_element_by_link_text("검색").click()

# 학습목표 1: 사용자가 입력한 건수와 실제 검색 건수를 비교 후 동기화하기
# Step 5. 사용자가 입력한 건수와 실제 검색 건수를 비교 후 동기화하기

cnt2=0

time.sleep(2)     # 페이지가 로딩 될 때까지 2초 기다립니다

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
search_cnt_1 = soup.find('div','total_check')
search_cnt_2 = search_cnt_1.find('strong').find('span').get_text()
    
if cnt > int(search_cnt_2) :
    cnt2 = int(search_cnt_2)
else :
    cnt2 = cnt

real_page_cnt = math.ceil(cnt2/10)

print("\n")
print("="*80)
print("요청하신 검색 건수는 %s 건이었고" %cnt)
print("해당 키워드로 실제 검색된 글의 건수가 총 %s 건이라서" %search_cnt_2)
print("실제 검색 건수는 %s 건으로 재설정하고" %cnt2)
print("총 %s 페이지의 게시글 수집을 진행하겠습니다" %real_page_cnt)
print("="*80)
print("\n")

#학습 목표 2: 페이지 이동하기
#Step 6. 페이지 번호를 바꾸면서 실제 데이터 추출하기

no = 1            # 게시글 번호 카운트용 변수

no2 =[ ]          # 게시글 번호 저장용 리스트
contents2=[ ]     # 게시글 내용 저장용 리스트
tags2=[ ]         # 해쉬 태그 정보 저장용 리스트
# 이중 for문 사용
for x in range(1,real_page_cnt+1) :  # 1페이지부터 반복 시작    
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    print("\n")    
    time.sleep(2)              #페이지가 로딩 될 때까지 2초 대기
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType flnon').find_all('div','area_txt')
    
    for i in content_list:  
        
        f = open(f_name, 'a',encoding='UTF-8')
        
        # 게시글 번호 입력
        no2.append(no)
        print('1.번호:',no)
        f.write("1.번호:"+str(no)+"\n")
    
        # 게시글의 내용 추출 및 입력
        contents = i.find('div','tit').get_text( )
        contents2.append(contents)
        print('2.요약내용:',contents.strip())
        f.write("2.요약내용:" + contents + "\n")
    
        # 학습목표 3: 예외 처리 사용하기
        try :
            tag = i.find('p','tag').get_text()
        except AttributeError :
            tag = '태그정보가 없습니다'
            tags2.append(tag)
            print('3.해쉬태그:',tag.strip())
            print("\n")       
            f.write("3.해쉬태그:"+tag+"\n")
            f.write("\n")
        else :
            tags2.append(tag)
            print('3.해쉬태그:',tag.strip())
            print("\n")
            f.write("3.해쉬태그:"+tag+"\n")
            f.write("\n")
            
            f.close()
        
        if no == cnt2 :
            break
    
        no += 1
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    x += 1
    
    if x == real_page_cnt+1 :   
        break
#for a in range(1,101):
#   driver.find_element_by_link_text('%s', %a).click()
    if x == 6 :
        driver.find_element_by_xpath("""//*[@id="6"]""").click()
    else :
        driver.find_element_by_link_text("""%s""" %x).click()     

#Step 7. 출력 결과를 표(데이터 프레임) 형태로 만들기

korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents2
korea['태그']=tags2


# 추출 결과를 csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 추출 결과를 엑셀 형태로 저장하기
korea.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %f_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

1.크롤링할 키워드는 무엇입니까?: 봄여행
2.크롤링 할 건수는 몇건입니까?: 25
3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test_1.txt
4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test_1.csv
5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\data\test_1.xls


요청하신 검색 건수는 25 건이었고
해당 키워드로 실제 검색된 글의 건수가 총 987 건이라서
실제 검색 건수는 25 건으로 재설정하고
총 3 페이지의 게시글 수집을 진행하겠습니다


1 페이지 내용 수집 시작합니다 =======================


1.번호: 1
2.요약내용: 강원도 '다시 봄!' - 봄 여행에 어울리는 강원도 숙소 모음
3.해쉬태그: #한국관광품질인증#여행기사#추천숙박#파로호한옥펜션#강릉오죽한옥마을#호텔탑스빌#MGM호텔#피카소호텔#화천#강릉#동해#바다#강원숙소#인증숙소#우수숙소#봄!_가족과_함께#강원도여행


1.번호: 2
2.요약내용: 마음을 나누는 엄마 딸 여행, 서울 남산 주변과 한강 물길 봄나들이
3.해쉬태그: #서울가볼만한곳#아이와함께#친구와함께#연인과함께#봄에가볼만한곳#충무로역#오재미동#영상예술#예술통#거리미술관#필동#남산골한옥마을#전통가옥#국악당#체험학습#전통문화체험#서울천년타임캡슐광장#남산케이블카#N서울타워#남산#한양도성성곽길#안중근의사기념관#남산도서관#서울여행코스#양화한강공원#한강#피크닉#나들이#봄나들이#선유도공원#카페나루#서울맛집#달강#서래섬#세빛섬#튜브스터#밤도깨비야시장#낭만달빛마켓#반포한강공원야시장#반포한강공원#서울밤도깨비야시장#스트리트뮤지엄#힐링#한강자전거


1.번호: 3
2.요약내용: 세계자연유산(UNESCO)의 봄을 느끼며, 제주도 동부 여행
3.해쉬태그: #장애인추천코스#제주가볼만한곳#제주당일코스#제주민속촌#제주올레길#제주해녀박물관#김녕어울림센터#월정리해변#제주돌문화공원#절물자연휴양림#메이즈랜드#두맹이골목#제주1박2일